In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # plotting library

print('Libraries imported.')

     |████████████████████████████████| 112 kB 4.1 MB/s eta 0:00:01
Libraries imported.


In [4]:
CLIENT_ID = 'OJ4HUXL1DFTO1JBWCJHVKOATZ4N1IKVB5ICSMCQPX4SOUUGH' # your Foursquare ID
CLIENT_SECRET = '5R4CYRHLWW0QXE1OYGO4GW5HF1WXN13DDACV1PTRYBS3BR1D' # your Foursquare Secret
ACCESS_TOKEN = 'MSHSK0QCV3W5AZZZ25NI5WPZAJB5B5JJNRRNOBW55C3CU1U3' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJ4HUXL1DFTO1JBWCJHVKOATZ4N1IKVB5ICSMCQPX4SOUUGH
CLIENT_SECRET:5R4CYRHLWW0QXE1OYGO4GW5HF1WXN13DDACV1PTRYBS3BR1D


In [17]:

address = 'Al Bateen St, Abu Dhabi, United Arab Emirates'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

24.4534476 54.3526753


In [22]:
search_query = 'Italian'
radius = 5000
print(search_query + ' .... OK!')

Italian .... OK!


In [23]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,514c800fe4b07ab0bfe80784,Bainuna St,AE,أبوظبي,الإمارات العربية المتحدة,NaN,159,"[Bainuna St, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45409759549625...",24.454098,54.351276,NaN,NaN,أبوظبي,Boccaccio Italian Dining Int. Hotel,v-1611210673,NaN
1,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",False,4dc8c398fa761aaccadfa71e,NaN,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1404,"[أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45636652957907...",24.456367,54.366160,NaN,NaN,أبوظبي,Italian Embassy,v-1611210673,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4d8065f370d18cfab9129c94,Sheikh Khalifa Street,AE,أبوظبي,الإمارات العربية المتحدة,NaN,4644,"[Sheikh Khalifa Street, أبوظبي, الإمارات العرب...","[{'label': 'display', 'lat': 24.49339441617108...",24.493394,54.365919,NaN,NaN,أبوظبي,Amalfi Fine Dining Italian Restaurant,v-1611210673,NaN
3,[],False,5b90fbe5dd70c5002c69cfea,Platinum Hotel Apartments Building Khalid Ibn ...,AE,أبوظبي,الإمارات العربية المتحدة,NaN,3027,[Platinum Hotel Apartments Building Khalid Ibn...,"[{'label': 'display', 'lat': 24.4805525868491,...",24.480553,54.355078,NaN,NaN,أبوظبي,Amood moroccan and Italian Restaurant,v-1611210673,NaN
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,5539fe77498e2ed1f12ab0f2,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3648,[الإمارات العربية المتحدة],"[{'label': 'display', 'lat': 24.48473942292588...",24.484739,54.363394,NaN,NaN,NaN,Crema/Italian cuisine,v-1611210673,NaN


In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Boccaccio Italian Dining Int. Hotel,Italian Restaurant,Bainuna St,AE,أبوظبي,الإمارات العربية المتحدة,NaN,159,"[Bainuna St, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45409759549625...",24.454098,54.351276,NaN,NaN,أبوظبي,514c800fe4b07ab0bfe80784
1,Italian Embassy,Embassy / Consulate,NaN,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1404,"[أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45636652957907...",24.456367,54.366160,NaN,NaN,أبوظبي,4dc8c398fa761aaccadfa71e
2,Amalfi Fine Dining Italian Restaurant,Restaurant,Sheikh Khalifa Street,AE,أبوظبي,الإمارات العربية المتحدة,NaN,4644,"[Sheikh Khalifa Street, أبوظبي, الإمارات العرب...","[{'label': 'display', 'lat': 24.49339441617108...",24.493394,54.365919,NaN,NaN,أبوظبي,4d8065f370d18cfab9129c94
3,Amood moroccan and Italian Restaurant,None,Platinum Hotel Apartments Building Khalid Ibn ...,AE,أبوظبي,الإمارات العربية المتحدة,NaN,3027,[Platinum Hotel Apartments Building Khalid Ibn...,"[{'label': 'display', 'lat': 24.4805525868491,...",24.480553,54.355078,NaN,NaN,أبوظبي,5b90fbe5dd70c5002c69cfea
4,Crema/Italian cuisine,Italian Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3648,[الإمارات العربية المتحدة],"[{'label': 'display', 'lat': 24.48473942292588...",24.484739,54.363394,NaN,NaN,NaN,5539fe77498e2ed1f12ab0f2
5,Seville's Italian Rest.,Italian Restaurant,Mellenium hotel,AE,أبوظبي,الإمارات العربية المتحدة,Khalifa street,4731,"[Mellenium hotel (Khalifa street), أبوظبي, الإ...","[{'label': 'display', 'lat': 24.494709, 'lng':...",24.494709,54.363890,NaN,NaN,أبوظبي,4bcde110fb84c9b6e9be223e
6,Peppino Italian Restaurant,Italian Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,4612,[الإمارات العربية المتحدة],"[{'label': 'display', 'lat': 24.49320607464761...",24.493206,54.365508,NaN,NaN,NaN,4f57872ce4b0a4bae73544b6
7,Mezzaluna Italian Restaurant,Italian Restaurant,Emirates Palace,AE,أبوظبي,الإمارات العربية المتحدة,NaN,3820,"[Emirates Palace, أبوظبي, الإمارات العربية الم...","[{'label': 'display', 'lat': 24.46259677601735...",24.462597,54.316338,NaN,NaN,أبوظبي,4de4ad32b0fb267148b9a499
8,Pappagallo Italian Restaurant,Italian Restaurant,Tourist Club Area,AE,أبوظبي,الإمارات العربية المتحدة,NaN,5834,"[Tourist Club Area, أبوظبي, الإمارات العربية ا...","[{'label': 'display', 'lat': 24.49872550766311...",24.498726,54.381675,NaN,46066,أبوظبي,4dd80a92e4cd37c893bb4994
9,Bocca,Italian Restaurant,Corniche St,AE,أبوظبي,الإمارات العربية المتحدة,Corniche Road شارع الكورنيش,2955,"[Corniche St (Corniche Road شارع الكورنيش), أب...","[{'label': 'display', 'lat': 24.46352825778917...",24.463528,54.325687,NaN,NaN,أبوظبي,4bd0051077b29c7448188a82


In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [26]:
venue_id = '4d8065f370d18cfab9129c94' # ID of Amalfi Fine Dining Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [30]:
venue_id = '4bd0051077b29c7448188a82' # ID of Bocca
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


In [32]:
venue_id = '4e691e3b18a89685777dd71a' # ID of Camacho
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.5


In [33]:
result['response']['venue']['tips']['count']

44

In [35]:
limit = 30 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()

tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]

pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"My new favorite place in AD! Probably, the best sushi @ AD. Their sushi is delicious, especially the blue dragon, a must-try! Loved their spicy (cooked) seafood salad too. Will surely come back again.",27,0,524c661f11d2466b01afa9b5,Dr. Sultan,NaN,33279608


In [36]:
import requests
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

results = requests.get(url).json()
'There are {} around the hotel.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around the hotel.'

In [37]:
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Joud Cafe,Café,Al Bateen,AE,أبوظبي,الإمارات العربية المتحدة,Khaleej Al Arabi St,431,"[Al Bateen (Khaleej Al Arabi St), أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.456658352186324, 'lng': 54.35505534315826}]",24.456658,54.355055,NaN,NaN,أبوظبي,57826940498e4d362fa37443
1,Lavazza,Café,Al Buttain,AE,أبوظبي,الإمارات العربية المتحدة,Alkhaleej al arabi,375,"[Al Buttain (Alkhaleej al arabi), أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45574870456958, 'lng': 54.35537966287776}]",24.455749,54.355380,NaN,NaN,أبوظبي,5742b9ca498e60c1d0b55b07
2,Cartel Coffee Roasters,Coffee Shop,469 Arabian Gult St.,AE,أبوظبي,الإمارات العربية المتحدة,Al Buteen,642,"[469 Arabian Gult St. (Al Buteen), أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.45817, 'lng': 54.356326}]",24.458170,54.356326,NaN,NaN,أبوظبي,5ce051d8b9ac38002c742988
3,Tasha's,African Restaurant,Unit B02 Cafe,AE,أبوظبي,الإمارات العربية المتحدة,Marsa Al Bateen,1315,"[Unit B02 Cafe (Marsa Al Bateen), أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.452857848794118, 'lng': 54.33970655119717}]",24.452858,54.339707,NaN,NaN,أبوظبي,58c02e7e30ecc66fc54f0a2c
4,The Abu Dhabi EDITION,Hotel,Al Bateen Marina,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1622,"[Al Bateen Marina, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.4519787, 'lng': 54.3367484}]",24.451979,54.336748,NaN,NaN,أبوظبي,5bd8b0d3446ea6002c611443
5,Switch,Restaurant,Marasi AlBateen,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1428,"[Marasi AlBateen, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.453515643625803, 'lng': 54.338580708377386}]",24.453516,54.338581,NaN,NaN,أبوظبي,58b0423e7ff1e40eb70dcae7
6,British Vets Centre,Pet Store,NaN,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1369,"[أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.464853902245252, 'lng': 54.34760040527272}]",24.464854,54.347600,NaN,NaN,أبوظبي,4dd53da645ddd7035528e13e
7,Bodytree,Yoga Studio,11th Street/Sheikh Hazza Bin Zayed.St and take the immediate first right. We are the front cream Villa directly across the road,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1404,"[11th Street/Sheikh Hazza Bin Zayed.St and take the immediate first right. We are the front cream Villa directly across the road, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.46241827602594, 'lng': 54.36242565450964}]",24.462418,54.362426,NaN,NaN,أبوظبي,504cb536e4b08bf9ea5b481d
8,Art House Cafe,Café,"Inside Etihad Modern Art Gallery, Villa 4/2, Street 4",AE,أبوظبي,الإمارات العربية المتحدة,Opposite Al Bateen Mall,886,"[Inside Etihad Modern Art Gallery, Villa 4/2, Street 4 (Opposite Al Bateen Mall), أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.460580439121916, 'lng': 54.34878235719427}]",24.460580,54.348782,Al Bateen,NaN,أبوظبي,53ff53b9498ef90189445fe5
9,Al Afiya Bath & Saloon,Spa,Behind Khalidiya Garden,AE,أبوظبي,الإمارات العربية المتحدة,NaN,1826,"[Behind Khalidiya Garden, أبوظبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 24.46898912704606, 'lng': 54.3469042941851}]",24.468989,54.346904,NaN,NaN,أبوظبي,4cebe003678aa09391f4faea


In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map